In [60]:
import pyvips as vips
def wrap_after_commas(s: str, N: int) -> str:
    if N <= 0: return s
    out, count = [], 0
    for ch in s:
        out.append(ch)
        count += 1
        if ch == "," and count >= N:
            out.append("\n")
            count = 0
    return "".join(out)


txt=(
    "slot:1,"
    "map:nn14:AB:-40:-40:40:40,"
    "m:j1s(cos(l**5-l**4+l**3-l**2+l))*j1s(ei(l*l-l))*j0s(si(l*x+l+x))*j1s(sin(l*l-l))*exp(aira(x)+1j*frec(l)),"
    "rgb:pfm:beauty_gucci_taupe_peach:pastel_warmwhite_forest:1.2:s1=0.868:s2=0.63:s3=0.977:se=1.0,"
    "hist:6:6:5120"
)

wrapped_txt = wrap_after_commas(txt,30)
print(wrapped_txt )

slot:1,map:nn14:AB:-40:-40:40:40,
m:j1s(cos(l**5-l**4+l**3-l**2+l))*j1s(ei(l*l-l))*j0s(si(l*x+l+x))*j1s(sin(l*l-l))*exp(aira(x)+1j*frec(l)),
rgb:pfm:beauty_gucci_taupe_peach:pastel_warmwhite_forest:1.2:s1=0.868:s2=0.63:s3=0.977:se=1.0,
hist:6:6:5120


In [61]:
def text2glyph(txt,max_chars):
    wrapped_txt = wrap_after_commas(txt,max_chars)
    glyph = vips.Image.text(wrapped_txt, dpi=300, font="PT Mono Bold", align="centre")
    bilevel_glyph = (glyph>254).ifthenelse(255, 0)
    return bilevel_glyph

In [63]:
glyph = text2glyph(txt,30)
print(f"width:{glyph.width},height:{glyph.height}")

width:3170,height:208


In [64]:
def scale_to_fit(overlay,base,ratio,pad):
    width_ratio = (base.width*(1-pad))/glyph.width
    wscaled_overlay = overlay.resize(width_ratio,kernel="nearest")
    height_ratio = min(1,base.height*ratio/wscaled_overlay.height)
    hscaled_overlay = wscaled_overlay.resize(height_ratio,kernel="nearest")
    return hscaled_overlay


In [65]:
def insertion_coords(base, overlay, ratio: float) -> tuple[int, int]:
    """
    Return (x, y) to insert `overlay` into `base` so it is:
      - horizontally centered
      - vertically centered within the bottom `ratio` fraction of `base` height

    ratio: 0..1 (e.g. 0.2 for bottom 20%)
    """
    H, W = int(base.height), int(base.width)
    h, w = int(overlay.height), int(overlay.width)

    r = float(ratio)
    if not (0.0 < r <= 1.0):
        raise ValueError("ratio must be in (0, 1].")

    band_h = max(1, int(round(r * H)))
    band_y0 = H - band_h

    x = (W - w) // 2
    y = band_y0 + (band_h - h) // 2
    return x, y

In [ ]:
def insert_glyph(base,overlay,ratio,pad):
    scaled_overlay = scale_to_fit(overlay,base,ratio,pad)
    x,y = insertion_coords(base,scaled_overlay, ratio)
    out = base.insert(scaled_overlay, x, y)
    return out


In [75]:
img = vips.Image.black(10000,10000).new_from_image(255)
txt=(
    "slot:1,"
    "map:nn14:AB:-40:-40:40:40,"
    "m:j1s(cos(l**5-l**4+l**3-l**2+l))*j1s(ei(l*l-l))*j0s(si(l*x+l+x))*j1s(sin(l*l-l))*exp(aira(x)+1j*frec(l)),"
    "rgb:pfm:beauty_gucci_taupe_peach:pastel_warmwhite_forest:1.2:s1=0.868:s2=0.63:s3=0.977:se=1.0,"
    "hist:6:6:5120"
)
glyph = text2glyph(txt,30)
insert_glyph(img,glyph,0.2,0.1).write_to_file("out.png")
